In [1]:
%tensorflow_version 2.x
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import re

TensorFlow 2.x selected.


In [0]:
df = pd.read_csv('/content/drive/My Drive/Accidental_Drug_Related_Deaths_2012-2018.csv')

In [3]:
df.head()

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
0,14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,substance,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,FAIRFIELD,Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Cocaine Intoxication,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,Hospital,NaN,Substance Abuse,Unknown,UNKNOWN,NaN,NaN,Acute Heroin and Cocaine Intoxication,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,NaN,Hospital,NaN,substance abuse,Residence,RYE,NaN,NaN,Acute Fentanyl and Morphine Intoxication,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,FAIRFIELD,Hospital,NaN,Transdermal Absorption,Other,NaN,NaN,NaN,Fentanyl Intoxication,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


The above table has a generic index, so we should set it to something more suitable if possible. The first column corresponds to ID, and we can readily check these values are unique using the `is_unique` method.

In [4]:
df['ID'].is_unique

True

In [0]:
# Set index to the ID column
df = df.set_index('ID')

In [6]:
df.head()

,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,substance,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,FAIRFIELD,Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Cocaine Intoxication,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,Hospital,NaN,Substance Abuse,Unknown,UNKNOWN,NaN,NaN,Acute Heroin and Cocaine Intoxication,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,NaN,Hospital,NaN,substance abuse,Residence,RYE,NaN,NaN,Acute Fentanyl and Morphine Intoxication,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,FAIRFIELD,Hospital,NaN,Transdermal Absorption,Other,NaN,NaN,NaN,Fentanyl Intoxication,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


We obviously do not need all the columns - as some contain less useful information such as the geo locations. Let us drop some of these columns.

In [0]:
cols_to_drop = ['DateType', 'ResidenceCity', 'ResidenceCounty', 
                'ResidenceState', 'LocationifOther', 'DescriptionofInjury', 
                'InjuryPlace', 'InjuryCity', 'InjuryCounty', 'InjuryState',
                'OtherSignifican', 'DeathCityGeo', 'ResidenceCityGeo', 
                'InjuryCityGeo']

In [0]:
df.drop(cols_to_drop, inplace=True, axis=1)

In [9]:
df.head()

,Date,Age,Sex,Race,DeathCity,DeathCounty,Location,COD,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
13-0102,03/21/2013 12:00:00 AM,48.0,Male,Black,NORWALK,FAIRFIELD,Hospital,Cocaine Intoxication,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
16-0165,03/13/2016 12:00:00 AM,30.0,Female,White,DANBURY,NaN,Hospital,Acute Heroin and Cocaine Intoxication,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
16-0208,03/31/2016 12:00:00 AM,23.0,Male,White,GREENWICH,NaN,Hospital,Acute Fentanyl and Morphine Intoxication,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
13-0052,02/13/2013 12:00:00 AM,22.0,Male,"Asian, Other",GREENWICH,FAIRFIELD,Hospital,Fentanyl Intoxication,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident


We can now neaten up our headers by renaming some of the poorly titled columns.

In [0]:
df.rename(columns = {'DeathCity':'City of Death', 
                     'DeathCounty':'County of Death',
                     'FentanylAnalogue':'Fentanyl Analogue',
                     'Morphine_NotHeroin':'Morphine',
                     'Amphet':'Amphetamine',
                     'Tramad':'Tramadol',
                     'AnyOpioid':'Any Opioid'},
           inplace = True) 

The `Date` column contains a complete timestamp but we are only interested in the date of death, not the time, we can use a regular expression to extract the date.

In [0]:
df['Date'] = df['Date'].str.extract(r'([0-9]{2}\/[0-9]{2}\/[0-9]{4})', expand=False)

Similarily the `Race` column contains elements which list more than one race, such as `Asian, Other`. We are only interested in the primary race, so we again use a regex to extract the first listed race.

In [0]:
df['Race'] = df['Race'].str.extract(r'([a-zA-Z]+),*', expand=False)

In [13]:
df.head(5)

,Date,Age,Sex,Race,City of Death,County of Death,Location,COD,Heroin,Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014,NaN,NaN,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
13-0102,03/21/2013,48.0,Male,Black,NORWALK,FAIRFIELD,Hospital,Cocaine Intoxication,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
16-0165,03/13/2016,30.0,Female,White,DANBURY,NaN,Hospital,Acute Heroin and Cocaine Intoxication,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
16-0208,03/31/2016,23.0,Male,White,GREENWICH,NaN,Hospital,Acute Fentanyl and Morphine Intoxication,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
13-0052,02/13/2013,22.0,Male,Asian,GREENWICH,FAIRFIELD,Hospital,Fentanyl Intoxication,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident


The columns containing the drugs in the deceased's blood have `NaN` if the drug was not present and `Y` if it was. Let us change this to a binary encoding using the `applymap` method and a lambda function to cast the values to 0 and 1.

We could do this by manually typing (or copying) all the column names, but that would be a little tedious. We could instead store all the column names into a list and use list comprehensions.

In [0]:
cols = list(df.columns)

In [15]:
cols

['Date',
 'Age',
 'Sex',
 'Race',
 'City of Death',
 'County of Death',
 'Location',
 'COD',
 'Heroin',
 'Cocaine',
 'Fentanyl',
 'Fentanyl Analogue',
 'Oxycodone',
 'Oxymorphone',
 'Ethanol',
 'Hydrocodone',
 'Benzodiazepine',
 'Methadone',
 'Amphetamine',
 'Tramadol',
 'Morphine',
 'Hydromorphone',
 'Other',
 'OpiateNOS',
 'Any Opioid',
 'MannerofDeath']

In [0]:
drug_cols = [cols[i] for i in range(cols.index('Heroin'),
                                    cols.index('Any Opioid') + 1)]

In [17]:
drug_cols

['Heroin',
 'Cocaine',
 'Fentanyl',
 'Fentanyl Analogue',
 'Oxycodone',
 'Oxymorphone',
 'Ethanol',
 'Hydrocodone',
 'Benzodiazepine',
 'Methadone',
 'Amphetamine',
 'Tramadol',
 'Morphine',
 'Hydromorphone',
 'Other',
 'OpiateNOS',
 'Any Opioid']

In [0]:
df[drug_cols] = df[drug_cols].applymap(lambda x: 1 if x=='Y' else 0)

In [19]:
df.head()

,Date,Age,Sex,Race,City of Death,County of Death,Location,COD,Heroin,Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014,NaN,NaN,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,Accident
13-0102,03/21/2013,48.0,Male,Black,NORWALK,FAIRFIELD,Hospital,Cocaine Intoxication,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident
16-0165,03/13/2016,30.0,Female,White,DANBURY,NaN,Hospital,Acute Heroin and Cocaine Intoxication,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
16-0208,03/31/2016,23.0,Male,White,GREENWICH,NaN,Hospital,Acute Fentanyl and Morphine Intoxication,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
13-0052,02/13/2013,22.0,Male,Asian,GREENWICH,FAIRFIELD,Hospital,Fentanyl Intoxication,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident


The data is looking much cleaner, but we still need to make sure we don't have any suprising values within the fields. This can be done by printing out all unique values for the columns with the `unique()` method.

In [20]:
df['Date'].unique()

array(['06/28/2014', '03/21/2013', '03/13/2016', ..., '12/26/2016',
       '12/02/2017', '09/15/2016'], dtype=object)

In [21]:
df['Age'].unique()

array([nan, 48., 30., 23., 22., 21., 40., 50., 26., 49., 29., 45., 42.,
       54., 32., 47., 39., 27., 34., 25., 51., 35., 43., 55., 56., 28.,
       67., 36., 52., 61., 53., 41., 31., 57., 24., 37., 38., 58., 59.,
       46., 64., 60., 44., 33., 20., 19., 66., 63., 62., 65., 71., 73.,
       68., 69., 75., 72., 18., 15., 70., 17., 74., 84., 14., 87., 81.,
       16.])

In [22]:
df['Race'].unique()

array([nan, 'Black', 'White', 'Asian', 'Hispanic', 'Unknown', 'Other',
       'Chinese', 'Native', 'Hawaiian'], dtype=object)

In [23]:
df['Sex'].unique()

array([nan, 'Male', 'Female', 'Unknown'], dtype=object)

In [24]:
df['City of Death'].unique()

array([nan, 'NORWALK', 'DANBURY', 'GREENWICH', 'BRISTOL', 'WEST HARTFORD',
       'NEW HAVEN', 'EAST HARTFORD', 'MONTVILLE', 'STRATFORD', 'HARTFORD',
       'WALLINGFORD', 'MIDDLETOWN', 'WINDHAM', 'BRIDGEPORT', 'DERBY',
       'COS COB', 'MERIDEN', 'ENFIELD', 'TORRINGTON', 'WATERBURY',
       'MASHANTUCKET', 'MILFORD', 'SOMERS', 'STAMFORD', 'NEW BRITAIN',
       'WILLIMANTIC', 'WATERTOWN', 'BALTIC', 'RIDGEFIELD', 'NEW LONDON',
       'BROOKFIELD', 'SHELTON', 'NORWICH', 'NAUGATUCK', 'VERNON',
       'FAIRFIELD', 'EAST HAMPTON', 'SHARON', 'MONROE', 'WINCHESTER',
       'BRANFORD', 'MANCHESTER', 'WOLCOTT', 'WEST SUFFIELD', 'FARMINGTON',
       'NORTH BRANFORD', 'NEW MILFORD', 'STONINGTON', 'PRESTON', 'GROTON',
       'SOUTHINGTON', 'SOUTH WINDSOR', 'JEWETT CITY', 'EAST WINDSOR',
       'OLD SAYBROOK', 'BROAD BROOK', 'PORTLAND', 'HEBRON', 'DARIEN',
       'LEDYARD', 'TRUMBULL', 'BURLINGTON', 'BLOOMFIELD', 'TERRYVILLE',
       'STAFFORD SPRINGS', 'WEST HAVEN', 'THOMPSON', 'OXFORD',
       '

In [25]:
df['County of Death'].unique()

array([nan, 'FAIRFIELD', 'HARTFORD', 'NEW HAVEN', 'NEW LONDON',
       'LITCHFIELD', 'MIDDLESEX', 'WINDHAM', 'TOLLAND', 'USA'],
      dtype=object)

In [26]:
df['Location'].unique()

array([nan, 'Hospital', 'Residence', 'Other', 'Nursing Home',
       'Convalescent Home', 'Hospice'], dtype=object)

In [27]:
df['MannerofDeath'].unique()

array(['Accident', 'accident', nan, 'Pending', 'ACCIDENT', 'Natural'],
      dtype=object)

We can see some columns contain a category for 'Unknown', but some entries are `NaN`, let us cast these `NaN` values to Unknown so we may better represent the data.

In [0]:
def clear_nan(item):
  if pd.isna(item):
    return 'Unknown'
  else:
    return item

In [0]:
nan_cols = [cols[i] for i in range(cols.index('Age'), 
                                   cols.index('Location') + 1)]

nan_cols.append('MannerofDeath')

In [0]:
df[nan_cols] = df[nan_cols].applymap(clear_nan)

In [38]:
df.head()

,Date,Age,Sex,Race,City of Death,County of Death,Location,COD,Heroin,Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"Acute fent, hydrocod, benzodiazepine",0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,Accident
13-0102,03/21/2013,48,Male,Black,NORWALK,FAIRFIELD,Hospital,Cocaine Intoxication,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident
16-0165,03/13/2016,30,Female,White,DANBURY,Unknown,Hospital,Acute Heroin and Cocaine Intoxication,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
16-0208,03/31/2016,23,Male,White,GREENWICH,Unknown,Hospital,Acute Fentanyl and Morphine Intoxication,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
13-0052,02/13/2013,22,Male,Asian,GREENWICH,FAIRFIELD,Hospital,Fentanyl Intoxication,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident


Those all caps strings in `City of Death` and `County of Death` don't look so great, let us format them appropriately. The string method `title()` formats a string such that its first character is upper case and the rest are lowercase. This will also sort out the duplicate forms of `Accident` in the `MannerofDeath` column.

In [0]:
df[['City of Death', 
    'County of Death',
    'MannerofDeath']] = df[['City of Death',
                            'County of Death',
                            'MannerofDeath']].applymap(lambda x: x.title())

In [40]:
df.head()

,Date,Age,Sex,Race,City of Death,County of Death,Location,COD,Heroin,Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"Acute fent, hydrocod, benzodiazepine",0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,Accident
13-0102,03/21/2013,48,Male,Black,Norwalk,Fairfield,Hospital,Cocaine Intoxication,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident
16-0165,03/13/2016,30,Female,White,Danbury,Unknown,Hospital,Acute Heroin and Cocaine Intoxication,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
16-0208,03/31/2016,23,Male,White,Greenwich,Unknown,Hospital,Acute Fentanyl and Morphine Intoxication,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
13-0052,02/13/2013,22,Male,Asian,Greenwich,Fairfield,Hospital,Fentanyl Intoxication,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident


In [41]:
df['MannerofDeath'].unique()

array(['Accident', 'Unknown', 'Pending', 'Natural'], dtype=object)

`Pending` implies we do not know the current manner of death, let us cast these values to `Unknown`.

In [0]:
df['MannerofDeath'] = df['MannerofDeath'].map(lambda x: 'Unknown' if x == 'Pending' else x)

In [43]:
df['MannerofDeath'].unique()

array(['Accident', 'Unknown', 'Natural'], dtype=object)

Finally, let us check if any of the Date columns contain `NaN` values and remove those rows. This can be done easily by taking the rows from the dateframe where `Date` is not `NaN`.

In [44]:
df['Date'].isna().sum()

2

In [0]:
# New dataframe containing only rows where 'Date' is not NaN
df = df[df['Date'].notna()]

In [46]:
df['Date'].isna().sum()

0

In [47]:
df.head()

,Date,Age,Sex,Race,City of Death,County of Death,Location,COD,Heroin,Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"Acute fent, hydrocod, benzodiazepine",0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,Accident
13-0102,03/21/2013,48,Male,Black,Norwalk,Fairfield,Hospital,Cocaine Intoxication,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident
16-0165,03/13/2016,30,Female,White,Danbury,Unknown,Hospital,Acute Heroin and Cocaine Intoxication,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
16-0208,03/31/2016,23,Male,White,Greenwich,Unknown,Hospital,Acute Fentanyl and Morphine Intoxication,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Accident
13-0052,02/13/2013,22,Male,Asian,Greenwich,Fairfield,Hospital,Fentanyl Intoxication,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Accident


Our data is now looking much better, we are ready to plot some visuals.